In [3]:
import os
import csv
import psycopg2

# Database connection details
dbname = 'mhdatabase'
user = 'postgres'
password = 'removed password!'
host = 'localhost'
port = '5432'

# Path to the folder containing the CSV files
csv_folder_path = '/SM-suicide_detection'

# List of CSV files
csv_files = [
    'addiction.csv',
    'adhd.csv',
    'anxiety.csv',
    'bipolar.csv',
    'bpd.csv',
    'depression.csv',
    'mentalhealth.csv',
    'ocd.csv',
    'ptsd.csv',
    'schizophrenia.csv',
    'selfharm.csv',
    'SuicideWatch.csv'
]

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cursor = conn.cursor()

# Loop through each CSV file
for csv_file in csv_files:
    table_name = csv_file[:-4]  # Remove the '.csv' extension to use as the table name
    file_path = os.path.join(csv_folder_path, csv_file)
    
    # Create the table with the required columns
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (post_date DATE, kind VARCHAR, subreddit VARCHAR, title VARCHAR, selftext VARCHAR);"
    cursor.execute(create_table_query)
    conn.commit()
    
    # Import data from the CSV file into the table
    with open(file_path, 'r') as file:
        csv_data = csv.reader(file)
        next(csv_data)  # Skip the header row
        insert_query = f"INSERT INTO {table_name} (post_date, kind, subreddit, title, selftext) VALUES (%s, %s, %s, %s, %s);"
        cursor.executemany(insert_query, csv_data)
        conn.commit()

# Close the database connection
cursor.close()
conn.close()
